In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [50]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os
import cv2
import gc
from tqdm import tqdm
from datetime import datetime
from typing import Optional
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt


from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold

from tensorflow import keras
import tensorflow as tf
import keras
from keras.models import load_model, save_model
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.losses import binary_crossentropy
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from keras import backend as K
from keras.models import Model
from keras.layers import Input

import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib as mpl
import os

In [51]:
BATCH_SIZE = 16
EPOCH = 10
n_splits = 5
fold_selected = 2

TRAIN_ROOT_DIR = "../input/uw-madison-gi-tract-image-segmentation/"
TEST_ROOT_DIR = "../input/uw-madison-gi-tract-image-segmentation/test/"

In [52]:
train_df_original = pd.read_csv(TRAIN_ROOT_DIR + 'train.csv')

print(train_df_original.shape)
train_df_original.head(30)

In [53]:
plt.figure(figsize=(28, 12))
train_df_original['class'].value_counts(normalize=True).plot.pie()

In [54]:
test_df = pd.read_csv(TRAIN_ROOT_DIR + 'sample_submission.csv')
test_df.head()

In [55]:
if len(test_df) == 0:
    DEBUG=True
    # test_df=train_df_original.iloc[:300, :]
    test_df=pd.read_csv(TRAIN_ROOT_DIR + 'train.csv').iloc[:300, :]
    test_df['segmentation'] = ''
    test_df = test_df.rename(columns={'segmentation' : 'prediction'})
else:
    DEBUG=False
    
submission = test_df.copy()
test_df.head()

In [56]:
train_df_original.head()

In [57]:
train_df_original.head()
train_df_original.shape

In [58]:
def df_preparation(df, subset='train', DEBUG=False) :
    df['case'] = df['id'].apply(lambda x : int(x.split('_')[0].replace('case', '')))
    df['day'] = df['id'].apply(lambda x : int(x.split('_')[1].replace('day', '')))
    df['slice'] = df['id'].apply(lambda x : x.split('_')[3])
    
    if(subset == 'train') or (DEBUG) :
        DIR = TRAIN_ROOT_DIR + "train"
    else :
        DIR = TEST_ROOT_DIR
        
    all_images = glob(os.path.join(DIR, "**", "*.png"), recursive=True)
    print("all_images length ", len(all_images))
    
    x = all_images[0].rsplit("/", 4)[0]
    print(x)
    path_partial_list = []
    for i in range(0, df.shape[0]):
        path_partial_list.append(
            os.path.join(
                x,
                "case" + str(df["case"].values[i]),
                "case"
                + str(df["case"].values[i])
                + "_"
                + "day"
                + str(df["day"].values[i]),
                "scans",
                "slice_" + str(df["slice"].values[i]),
            )
        )
    df["path_partial"] = path_partial_list
    path_partial_list = []
    for i in range(0, len(all_images)):
        path_partial_list.append(str(all_images[i].rsplit("_", 4)[0]))

    tmp_df = pd.DataFrame()
    tmp_df["path_partial"] = path_partial_list
    tmp_df["path"] = all_images

    df = df.merge(tmp_df, on="path_partial").drop(columns=["path_partial"])

    df["width"] = df["path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[1]))
    df["height"] = df["path"].apply(lambda x: int(x[:-4].rsplit("_", 4)[2]))

    del x, path_partial_list, tmp_df

    return df


In [59]:
train_df = df_preparation(train_df_original, subset='train')
train_df.head(10)

In [60]:
print(train_df['path'][0])
print(train_df['path'][1])
print(train_df['path'][2])
print(train_df['path'][3])

In [61]:
test_df.head()
test_df.shape

In [62]:
test_df=df_preparation(test_df, subset="test", DEBUG=True)
test_df.head()

In [63]:
test_df['path'][0]

In [64]:
def df_rearrange_for_3_segmentation_classes(df, subset="train"):
    df_restructured = pd.DataFrame({"id": df["id"][::3]})

    if subset == "train":
        df_restructured["large_bowel"] = df["segmentation"][::3].values
        df_restructured["small_bowel"] = df["segmentation"][1::3].values
        df_restructured["stomach"] = df["segmentation"][2::3].values

    df_restructured["path"] = df["path"][::3].values
    df_restructured["case"] = df["case"][::3].values
    df_restructured["day"] = df["day"][::3].values
    df_restructured["slice"] = df["slice"][::3].values
    df_restructured["width"] = df["width"][::3].values
    df_restructured["height"] = df["height"][::3].values

    df_restructured = df_restructured.reset_index(drop=True)
    df_restructured = df_restructured.fillna("")
    if subset == "train":
        df_restructured["count"] = np.sum(
            df_restructured.iloc[:, 1:4] != "", axis=1
        ).values

    return df_restructured


In [65]:
train_df_rearranged=df_rearrange_for_3_segmentation_classes(train_df, subset="train")
train_df_rearranged.head(100)

In [66]:
train_df_rearranged = train_df_rearranged[(train_df['case']!=7)|(train_df['day']!=0)].reset_index(drop=True)

train_df_rearranged = train_df_rearranged[(train_df['case']!=81)|(train_df['day']!=30)].reset_index(drop=True)

train_df_rearranged = train_df_rearranged[(train_df['case']!=138)|(train_df['day']!=00)].reset_index(drop=True)

In [67]:
gc.collect()

In [68]:
def plot_bar(df):
    plt.figure(figsize=(12, 6))
    bar = plt.bar([1, 2, 3], 100 * np.mean(df.iloc[:, 1:4] != "", axis=0))
    plt.title("Percent Training Images with Mask", fontsize=16)
    plt.ylabel("Percent of Train images with mask")
    plt.xlabel("Class Types")
    # labels = ["large bowel", "small bowel", "stomach"]
    labels = ["large_bowel", "small_bowel", "stomach"]

    for rect, lbl in zip(bar, labels):
        height = rect.get_height()
        plt.text(
            rect.get_x() + rect.get_width() / 3,
            height,
            lbl,
            ha="center",
            va="bottom",
            fontsize=12,
        )

    plt.ylim((0, 50))
    plt.show()


In [69]:
plot_bar(train_df_rearranged)

In [70]:
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import cv2

import tensorflow as tf



class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size=BATCH_SIZE, subset="train", shuffle=False):
        super().__init__()
        self.df = df
        self.shuffle = shuffle
        self.subset = subset
        self.batch_size = batch_size
        self.indexes = np.arange(len(df))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    """ __getitem__ returns a batch of images and masks """

    def __getitem__(self, index):
        X = np.empty((self.batch_size, 128, 128, 3))  # Makes a 4-D Tensor
        y = np.empty((self.batch_size, 128, 128, 3))  # Makes a 4-D Tensor

        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]

        for i, img_path in enumerate(self.df["path"].iloc[indexes]):
            # print("df['path'].iloc[indexes].shape ", self.df['path'].iloc[indexes].shape) # (16,)
            # in above 'i' is just the counter. i.e. starts from 0 and goes upto the max length of all the rows
            w = self.df["width"].iloc[
                indexes[i]
            ]  # selects the row number of indexes[i]
            h = self.df["height"].iloc[indexes[i]]
            img = self._load_grayscaled_img(img_path)  # shape: (128,128,1)
            # print('img shape after _load_grayscaled_img ', img.shape) #(128, 128, 1)
            # Now update X[i,] to be this image.
            X[
                i,
            ] = img  # broadcast to shape: (128,128,3)
            # As we know, that arr[1,] is equivalent to arr[1, :]
            # As NumPy will automatically insert trailing slices for you

            # print('X after ', X.shape) # (16, 128, 128, 3)
            # The slice notation in the above line means -
            # Set me the (i+1)th Row of X to be this image

            if self.subset == "train":
                for k, j in enumerate(["large_bowel", "small_bowel", "stomach"]):
                    # Now 'j' will take each value from the above list
                    # e.g. self.df['large_bowel']
                    # and in my train_df_rearranged each of the ["large_bowel","small_bowel","stomach"]
                    # column names contain RLE formatted segmentation data.
                    rles = self.df[j].iloc[indexes[i]]
                    # so the above line will actually be something like => self.df['stomach'].iloc[indexes[20]]
                    # giving me the RLE data for that row and column
                    # mask = rle_decode(rles, shape=(h, w, 1))
                    # if all my utils method is in separate file then uncomment below
                    mask = rle_decode(rles, shape=(h, w, 1))
                    mask = cv2.resize(mask, (128, 128))
                    y[i, :, :, k] = mask
        if self.subset == "train":
            return X, y
        else:
            return X

    def _load_grayscaled_img(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_ANYDEPTH)
        img_size = (128, 128)
        img = cv2.resize(img, img_size)
        img = img.astype(np.float32) / 255.0
        img = np.expand_dims(img, axis=-1)
        return img

    """cv2.IMREAD_ANYDEPTH => If set, return 16-bit/32-bit image when the input has the corresponding depth, otherwise convert it to 8-bit. """

In [102]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle_decode(mask_rle, shape, color=1):
    s = mask_rle.split()
    starts, length = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + length
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = color
    return img.reshape(shape)


def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def dice_loss(y_true, y_pred):
    smooth = 1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(tf.cast(y_true, tf.float32), y_pred) + dice_loss(tf.cast(y_true, tf.float32), y_pred)

In [103]:
def plot_mask_with_color_patches(df, colors, labels):
    list_indices_of_mask_random = list(
        df[df["large_bowel"] != ""].sample(BATCH_SIZE).index
    )
    list_indices_of_mask_random += list(
        df[df["small_bowel"] != ""].sample(BATCH_SIZE * 2).index
    )
    list_indices_of_mask_random += list(
        df[df["stomach"] != ""].sample(BATCH_SIZE * 3).index
    )
    # print('list_indices_of_mask_random ', list_indices_of_mask_random)
    # It will be a list of indexes like [15176, 13709, 30423, ..., 12730]

    batches_from_datagen = DataGenerator(
        df[df.index.isin(list_indices_of_mask_random)], shuffle=True
    )

    num_rows = 6

    fig = plt.figure(figsize=(10, 25))
    gs = gridspec.GridSpec(nrows=num_rows, ncols=2)
    patches = [
        mpatches.Patch(color=colors[i], label=f"{labels[i]}")
        for i in range(len(labels))
    ]

    cmap1 = mpl.colors.ListedColormap(colors[0])
    cmap2 = mpl.colors.ListedColormap(colors[1])
    cmap3 = mpl.colors.ListedColormap(colors[2])
    """ The `matplotlib.colors.ListedColormap` class is used to create colarmap objects from a list of colors.
    The class belongs to the `matplotlib.colors` module. This module is used for converting color or numbers arguments to RGBA or RGB and for mapping numbers to colors or color specification conversion in a 1-D array of colors also known as colormap.
    This can be useful for directly indexing into colormap and it can also be used to create special colormaps for normal mapping. """

    for i in range(num_rows):
        images, mask = batches_from_datagen[i]
        # print('images.shape ', images.shape) # (16, 128, 128, 3)
        # print('mask.shape ', mask.shape) # (16, 128, 128, 3)
        """
        For each ID, we are going to create an image of shape [img height, img width, 3], where 3 (number of channels) are the 3 layers for each class:
        * the first layer: large bowel
        * the second layer: small bowel
        * the third layer: stomach
        """
        sample_img = images[0, :, :, 0]  # After this the shapes will be (128, 128)
        mask1 = mask[0, :, :, 0]  # After this the shapes will be (128, 128)
        mask2 = mask[0, :, :, 1]  # After this the shapes will be (128, 128)
        mask3 = mask[0, :, :, 2]  # After this the shapes will be (128, 128)

        ax0 = fig.add_subplot(gs[i, 0])  # i here is the row-counter which is 6
        im = ax0.imshow(sample_img, cmap="bone")

        ax1 = fig.add_subplot(gs[i, 1])
        if i == 0:
            ax0.set_title("Image", fontsize=15, weight="bold", y=1.02)
            ax1.set_title("Mask", fontsize=15, weight="bold", y=1.02)
            plt.legend(
                handles=patches,
                bbox_to_anchor=(1.1, 0.65),
                loc=2,
                borderaxespad=0.4,
                fontsize=14,
                title="Mask Labels",
                title_fontsize=14,
                edgecolor="black",
                facecolor="#c5c6c7",
            )

        # print('mask1 ', mask1.shape) # (128, 128)
        # print('mask2 ', mask2.shape) # (128, 128)
        # print('mask3 ', mask3.shape) # (128, 128)
        # print('np.ma.masked_where(mask1== False,  mask1) ', np.ma.masked_where(mask1== True,  mask1))
        l0 = ax1.imshow(sample_img, cmap="bone")
        l1 = ax1.imshow(np.ma.masked_where(mask1 == False, mask1), cmap=cmap1, alpha=1)
        l2 = ax1.imshow(np.ma.masked_where(mask2 == False, mask2), cmap=cmap2, alpha=1)
        l3 = ax1.imshow(np.ma.masked_where(mask3 == False, mask3), cmap=cmap3, alpha=1)
        # l1 = ax1.imshow(np.ma.masked_where(mask1== 0,  mask1),cmap=cmap1, alpha=1)
        # l2 = ax1.imshow(np.ma.masked_where(mask2== 0,  mask2),cmap=cmap2, alpha=1)
        # l3 = ax1.imshow(np.ma.masked_where(mask3== 0,  mask3),cmap=cmap3, alpha=1)
        _ = [ax.set_axis_off() for ax in [ax0, ax1]]

        colors = [im.cmap(im.norm(1)) for im in [l1, l2, l3]]

In [73]:
colors = ['blue', 'green', 'red']
labels = ['large_bowel', 'small_bowel', 'stomach']
plot_mask_with_color_patches(train_df_rearranged, colors, labels)

In [74]:
skf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (_, val_idx) in enumerate(
    skf.split(
        X=train_df_rearranged,
        y=train_df_rearranged["count"],
        groups=train_df_rearranged["case"],
    ),
    1,
):

    train_df_rearranged.loc[val_idx, 'fold'] = fold
    
train_df_rearranged["fold"] = train_df_rearranged["fold"].astype(np.uint8)

train_ids = train_df_rearranged[train_df_rearranged["fold"] != fold_selected].index
valid_ids = train_df_rearranged[train_df_rearranged["fold"] == fold_selected].index

X_train = train_df_rearranged[train_df_rearranged.index.isin(train_ids)]
X_valid = train_df_rearranged[train_df_rearranged.index.isin(valid_ids)]

train_df_rearranged.groupby("fold").size()

In [75]:
train_df_rearranged.head()

In [76]:
train_df_rearranged.groupby(['fold', 'count'])['id'].count()

In [77]:
experiment = False
if experiment:
    X_train = X_train[X_train.case.isin(X_train.case.unique()[:5])]
    X_valid = X_valid[X_valid.case.isin(X_valid.case.unique()[:2])]
    
print(X_train.shape)
print(X_valid.shape)

In [78]:
X_train

In [79]:
X_valid

In [80]:
train_generator = DataGenerator(X_train, shuffle = True)
val_generator = DataGenerator(X_valid)

In [81]:
train_generator

In [82]:
 pip install segmentation-models

In [83]:
! pip install git+https://github.com/qubvel/segmentation_models

In [84]:
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

In [85]:
from segmentation_models import Unet
from segmentation_models.utils import set_trainable

model = Unet('inceptionresnetv2', input_shape=(128, 128, 3), classes=3, activation='sigmoid', encoder_weights = 'imagenet' )
model.compile(optimizer = 'adam', loss=bce_dice_loss, metrics=[dice_coef, iou_coef])

In [86]:
checkpoint = ModelCheckpoint(
    'UNET_model',
    monitor = 'val_loss',
    verbose=1,
    save_best_only=True,
    mode = 'auto'
)

early_stopping = EarlyStopping(
    patience = 5,
    min_delta = 0.0001,
    restore_best_weights= True
)

lr_plateau = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=5,
    verbose=0,
    min_delta=0.0001,
)

In [87]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint, early_stopping],
    use_multiprocessing=False,
    workers=4,
    epochs=EPOCH
)

In [88]:
history_df = pd.DataFrame(history.history)
history_df.to_csv('history_df')

In [89]:
plt.figure(figsize=(20, 20))
plt.subplot(1, 3, 1)
plt.plot(range(history.epoch[-1] + 1), history.history['loss'], label='Train Loss' )
plt.plot(range(history.epoch[-1] + 1), history.history['val_loss'], label='Validation Loss' )
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Losses')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(range(history.epoch[-1] + 1), history.history['dice_coef'], label='Train Dice Coeff' )
plt.plot(range(history.epoch[-1] + 1), history.history['val_dice_coef'], label='Validation Dice Coef' )
plt.title('Dice Loss')
plt.xlabel('Epochs')
plt.ylabel('Dice Coef')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(range(history.epoch[-1] + 1), history.history['iou_coef'], label='Train IoU Coeff' )
plt.plot(range(history.epoch[-1] + 1), history.history['val_iou_coef'], label='Validation IoU Coef' )
plt.title('IoU Loss')
plt.xlabel('Epochs')
plt.ylabel('IoU Coef')
plt.legend()
plt.show()

In [90]:
pred_batches = DataGenerator(X_valid.iloc[200:208, :], batch_size=1, subset='train', shuffle=True)
preds = model.predict(pred_batches, verbose=1)

In [91]:
Threshold = 0.5

fig = plt.figure(figsize =(10, 25))
gs = gridspec.GridSpec(nrows = 8, ncols = 3)
colors = ['yellow','green','red']
labels = ["Large Bowel", "Small Bowel", "Stomach"]

patches = [mpatches.Patch(color=colors[i], label=f"{labels[i]}") for i in range(len(labels))]

cmap1 = mpl.colors.ListedColormap(colors[0])
cmap2 = mpl.colors.ListedColormap(colors[1])
cmap3 = mpl.colors.ListedColormap(colors[2])

for i in range(8):
    images, mask = pred_batches[i]
    sample_img = images[0, :, :, 0]
    
    mask1 = mask[0, :, :, 0]
    mask2 = mask[0, :, :, 1]
    mask3 = mask[0, :, :, 2]
    
    prediction_i = preds[i]
    predict1 = prediction_i[:, :, 0]
    predict2 = prediction_i[:, :, 1]
    predict3 = prediction_i[:, :, 2]
    
    predict1 = (predict1 > Threshold).astype(np.float32)
    predict2 = (predict2 > Threshold).astype(np.float32)
    predict3 = (predict3 > Threshold).astype(np.float32)
    
    ax0 = fig.add_subplot(gs[i, 0])
    im = ax0.imshow(sample_img, cmap='bone')
    ax0.set_title("Image", fontsize=12, y=1.01)
    
    ax1 = fig.add_subplot(gs[i, 1])
    ax1.set_title("Mask", fontsize=12, y=1.01)
    l0 = ax1.imshow(sample_img, cmap="bone")
    l1 = ax1.imshow(np.ma.masked_where(mask1 == False, mask1), cmap=cmap1, alpha=1)
    l2 = ax1.imshow(np.ma.masked_where(mask2 == False, mask2), cmap=cmap2, alpha=1)
    l3 = ax1.imshow(np.ma.masked_where(mask3 == False, mask3), cmap=cmap3, alpha=1)
    
    ax2 = fig.add_subplot(gs[i, 2])
    ax2.set_title("Predict", fontsize=12, y=1.01)
    l0 = ax1.imshow(sample_img, cmap="bone")
    l1 = ax1.imshow(np.ma.masked_where(predict1 == False, predict1), cmap=cmap1, alpha=1)
    l2 = ax1.imshow(np.ma.masked_where(predict2 == False, predict2), cmap=cmap2, alpha=1)
    l3 = ax1.imshow(np.ma.masked_where(predict3 == False, predict3), cmap=cmap3, alpha=1)
    
    _ = [ax.set_axis_off() for ax in [ax0, ax1, ax2]]
    colors = [im.cmap(im.norm(1)) for im in [l1, l2, l3] ]
    plt.legend(handles=patches, bbox_to_anchor=(1.1, 0.65), loc=2, borderaxespad=0.4, fontsize=12, title="Mask Labels", title_fontsize=12, edgecolor='black', facecolor='#c5c6c7' )
    
    

In [104]:
pred_batches = DataGenerator(test_df, batch_size = BATCH_SIZE, subset="test", shuffle=False)
num_batches = int(len(test_df)/BATCH_SIZE)

for i in range(num_batches):
    # Predict
    preds = model.predict(pred_batches[i],verbose=0)     # shape: (16,128,128,3)
    
    # Rle encode
    for j in range(BATCH_SIZE):
        for k in range(3):
            pred_img = cv2.resize(preds[j,:,:,k], (test_df.loc[i*BATCH_SIZE+j,"width"], test_df.loc[i*BATCH_SIZE+j,"height"]), interpolation=cv2.INTER_NEAREST) # resize probabilities to original shape
            pred_img = (pred_img>0.5).astype(dtype='uint8')    # classify
            submission.loc[3*(i*BATCH_SIZE+j)+k,'predicted'] = rle_encode(pred_img)

In [105]:
submission.to_csv('submission.csv', index=False)
submission.sample(20)